In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mydataset/data_train.csv


In [2]:
from numpy import array
from numpy import hstack
from keras.models import Sequential
from keras.layers import LSTM, Dense, Flatten, Bidirectional, TimeDistributed, Conv1D, MaxPooling1D, RepeatVector
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
import math
from sklearn.preprocessing import StandardScaler
import csv

2025-05-11 01:33:14.242128: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746927194.519090      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746927194.593360      13 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
df = pd.read_csv('/kaggle/input/mydataset/data_train.csv')

df = df.sort_values(by=['company_id', 'quarter'])

In [4]:
scaler = StandardScaler()

X = df[['P', 'EPS', 'D/E', 'SIZE']]

X_scaled = scaler.fit_transform(X)

df_scaled = pd.DataFrame(X_scaled, columns=['P', 'EPS', 'D/E', 'SIZE'])

df_scaled['quarter'] = df['quarter'].values
df_scaled['company_id'] = df['company_id'].values

df_scaled.to_csv('data_scaled.csv', index=False)

In [5]:
def split_sequence(sequence, n_steps_in, n_steps_out):
    X, y = list(), list()
    for i in range(len(sequence)):

        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out

        if out_end_ix > len(sequence):
            break

        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix:out_end_ix][:, 0]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)


In [6]:
df = pd.read_csv('/kaggle/working/data_scaled.csv')

df = df.sort_values(by=['company_id', 'quarter'])

company_ids = df['company_id'].unique()

In [7]:
def train_double_stacked_lstm(n_steps_in, n_steps_out, activation, loss, epochs):
    X_list, y_list = list(), list()

    for company_id in company_ids:
        company_data = df[df['company_id'] == company_id]
    
        p = array(company_data['P']).reshape((len(company_data), 1))
        eps = array(company_data['EPS']).reshape((len(company_data), 1))
        de = array(company_data['D/E']).reshape((len(company_data), 1))
        size = array(company_data['SIZE']).reshape((len(company_data), 1))
    
        dataset = hstack((p, eps, de, size))
    
        X, y = split_sequence(dataset, n_steps_in, n_steps_out)

        X_list.extend(X)
        y_list.extend(y)

    X = array(X_list)
    y = array(y_list)
    
    n_features = X.shape[2]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=True)

    model = Sequential()
    model.add(LSTM(100, activation=activation, return_sequences=True, input_shape=(n_steps_in, n_features)))
    model.add(LSTM(100, activation=activation))
    model.add(Dense(n_steps_out))
    model.compile(optimizer='adam', loss=loss)

    model.fit(X_train, y_train, epochs=epochs, verbose=0)

    loss = model.evaluate(X_test, y_test, verbose=0)
    
    return loss

In [8]:
def train_triple_stacked_lstm(n_steps_in, n_steps_out, activation, loss, epochs):
    X_list, y_list = list(), list()

    for company_id in company_ids:
        company_data = df[df['company_id'] == company_id]
    
        p = array(company_data['P']).reshape((len(company_data), 1))
        eps = array(company_data['EPS']).reshape((len(company_data), 1))
        de = array(company_data['D/E']).reshape((len(company_data), 1))
        size = array(company_data['SIZE']).reshape((len(company_data), 1))
    
        dataset = hstack((p, eps, de, size))
    
        X, y = split_sequence(dataset, n_steps_in, n_steps_out)

        X_list.extend(X)
        y_list.extend(y)

    X = array(X_list)
    y = array(y_list)
    
    n_features = X.shape[2]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=True)

    model = Sequential()
    model.add(LSTM(100, activation=activation, return_sequences=True, input_shape=(n_steps_in, n_features)))
    model.add(LSTM(100, activation=activation, return_sequences=True, input_shape=(n_steps_in, n_features)))
    model.add(LSTM(100, activation=activation))
    model.add(Dense(n_steps_out))
    model.compile(optimizer='adam', loss=loss)

    model.fit(X_train, y_train, epochs=epochs, verbose=0)

    loss = model.evaluate(X_test, y_test, verbose=0)
    
    return loss

In [9]:
def train_bidirectional_lstm(n_steps_in, n_steps_out, activation, loss, epochs):
    X_list, y_list = list(), list()

    for company_id in company_ids:
        company_data = df[df['company_id'] == company_id]
    
        p = array(company_data['P']).reshape((len(company_data), 1))
        eps = array(company_data['EPS']).reshape((len(company_data), 1))
        de = array(company_data['D/E']).reshape((len(company_data), 1))
        size = array(company_data['SIZE']).reshape((len(company_data), 1))
    
        dataset = hstack((p, eps, de, size))
    
        X, y = split_sequence(dataset, n_steps_in, n_steps_out)

        X_list.extend(X)
        y_list.extend(y)

    X = array(X_list)
    y = array(y_list)
    
    n_features = X.shape[2]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=True)

    model = Sequential()
    model.add(Bidirectional(LSTM(100, activation=activation), input_shape=(n_steps_in, n_features)))
    model.add(Dense(n_steps_out))
    model.compile(optimizer='adam', loss=loss)

    model.fit(X_train, y_train, epochs=epochs, verbose=0)

    loss = model.evaluate(X_test, y_test, verbose=0)
    
    return loss

In [10]:
def train_cnn_lstm(n_steps_in, n_seq, n_steps_out, activation, loss, epochs):
    X_list, y_list = list(), list()

    for company_id in company_ids:
        company_data = df[df['company_id'] == company_id]
    
        p = array(company_data['P']).reshape((len(company_data), 1))
        eps = array(company_data['EPS']).reshape((len(company_data), 1))
        de = array(company_data['D/E']).reshape((len(company_data), 1))
        size = array(company_data['SIZE']).reshape((len(company_data), 1))
    
        dataset = hstack((p, eps, de, size))
    
        X, y = split_sequence(dataset, n_steps_in * n_seq, n_steps_out)

        X_list.extend(X)
        y_list.extend(y)

    X = array(X_list)
    y = array(y_list)
    
    n_features = X.shape[2]
    
    X = X.reshape((X.shape[0], n_seq, n_steps_in, n_features))

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=True)

    model = Sequential()
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=1, activation=activation), input_shape=(None, n_steps_in, n_features)))
    model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
    model.add(TimeDistributed(Flatten()))
    model.add(LSTM(100, activation=activation))
    model.add(Dense(n_steps_out))
    model.compile(optimizer='adam', loss=loss)

    model.fit(X_train, y_train, epochs=epochs, verbose=0)

    loss = model.evaluate(X_test, y_test, verbose=0)
    
    return loss

In [11]:
def train_bidirectional_stacked_lstm(n_steps_in, n_steps_out, activation, loss, epochs):
    X_list, y_list = list(), list()

    for company_id in company_ids:
        company_data = df[df['company_id'] == company_id]
    
        p = array(company_data['P']).reshape((len(company_data), 1))
        eps = array(company_data['EPS']).reshape((len(company_data), 1))
        de = array(company_data['D/E']).reshape((len(company_data), 1))
        size = array(company_data['SIZE']).reshape((len(company_data), 1))
    
        dataset = hstack((p, eps, de, size))
    
        X, y = split_sequence(dataset, n_steps_in, n_steps_out)

        X_list.extend(X)
        y_list.extend(y)

    X = array(X_list)
    y = array(y_list)
    
    n_features = X.shape[2]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=True)

    model = Sequential()
    model.add(LSTM(100, activation=activation, return_sequences=True, input_shape=(n_steps_in, n_features)))
    model.add(Bidirectional(LSTM(100, activation=activation)))
    model.add(Dense(n_steps_out))
    model.compile(optimizer='adam', loss=loss)

    model.fit(X_train, y_train, epochs=epochs, verbose=0)

    loss = model.evaluate(X_test, y_test, verbose=0)
    
    return loss

In [12]:
def train_bidirectional_cnn_lstm(n_steps_in, n_seq, n_steps_out, activation, loss, epochs):
    X_list, y_list = list(), list()

    for company_id in company_ids:
        company_data = df[df['company_id'] == company_id]
    
        p = array(company_data['P']).reshape((len(company_data), 1))
        eps = array(company_data['EPS']).reshape((len(company_data), 1))
        de = array(company_data['D/E']).reshape((len(company_data), 1))
        size = array(company_data['SIZE']).reshape((len(company_data), 1))
    
        dataset = hstack((p, eps, de, size))
    
        X, y = split_sequence(dataset, n_steps_in * n_seq, n_steps_out)

        X_list.extend(X)
        y_list.extend(y)

    X = array(X_list)
    y = array(y_list)
    
    n_features = X.shape[2]
    
    X = X.reshape((X.shape[0], n_seq, n_steps_in, n_features))

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=True)

    model = Sequential()
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=1, activation=activation), input_shape=(None, n_steps_in, n_features)))
    model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
    model.add(TimeDistributed(Flatten()))
    model.add(Bidirectional(LSTM(100, activation=activation)))
    model.add(Dense(n_steps_out))
    model.compile(optimizer='adam', loss=loss)

    model.fit(X_train, y_train, epochs=epochs, verbose=0)

    loss = model.evaluate(X_test, y_test, verbose=0)
    
    return loss

In [13]:
activations = ['gelu', 'relu', 'linear']
losses = ['mse', 'mape']

In [14]:
with open('results_double_stacked_lstm.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['a', 'b', 'activation', 'loss_function', 'final_loss'])  # Header

    for a in range(1, 6):
        for b in range(1, 6):
            for act in activations:
                for loss_fn in losses:
                    print(f"Running: a={a}, b={b}, activation={act}, loss={loss_fn}")
                    final_loss = train_double_stacked_lstm(a, b, act, loss_fn, 200)
                    writer.writerow([a, b, act, loss_fn, final_loss])

Running: a=1, b=1, activation=gelu, loss=mse


2025-05-11 01:33:31.740224: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Running: a=1, b=1, activation=gelu, loss=mape
Running: a=1, b=1, activation=relu, loss=mse
Running: a=1, b=1, activation=relu, loss=mape
Running: a=1, b=1, activation=linear, loss=mse
Running: a=1, b=1, activation=linear, loss=mape
Running: a=1, b=2, activation=gelu, loss=mse
Running: a=1, b=2, activation=gelu, loss=mape
Running: a=1, b=2, activation=relu, loss=mse
Running: a=1, b=2, activation=relu, loss=mape
Running: a=1, b=2, activation=linear, loss=mse
Running: a=1, b=2, activation=linear, loss=mape
Running: a=1, b=3, activation=gelu, loss=mse
Running: a=1, b=3, activation=gelu, loss=mape
Running: a=1, b=3, activation=relu, loss=mse
Running: a=1, b=3, activation=relu, loss=mape
Running: a=1, b=3, activation=linear, loss=mse
Running: a=1, b=3, activation=linear, loss=mape
Running: a=1, b=4, activation=gelu, loss=mse
Running: a=1, b=4, activation=gelu, loss=mape
Running: a=1, b=4, activation=relu, loss=mse
Running: a=1, b=4, activation=relu, loss=mape
Running: a=1, b=4, activation=li

In [15]:
# with open('results_triple_stacked_lstm.csv', mode='w', newline='') as file:
#     writer = csv.writer(file)
#     writer.writerow(['a', 'b', 'activation', 'loss_function', 'final_loss'])  # Header

#     for a in range(1, 4):
#         for b in range(1, 3):
#             for act in activations:
#                 for loss_fn in losses:
#                     print(f"Running: a={a}, b={b}, activation={act}, loss={loss_fn}")
#                     final_loss = train_triple_stacked_lstm(a, b, act, loss_fn, 200)
#                     writer.writerow([a, b, act, loss_fn, final_loss])

In [16]:
# with open('results_bidirectional_lstm.csv', mode='w', newline='') as file:
#     writer = csv.writer(file)
#     writer.writerow(['a', 'b', 'activation', 'loss_function', 'final_loss'])  # Header

#     for a in range(1, 4):
#         for b in range(1, 3):
#             for act in activations:
#                 for loss_fn in losses:
#                     print(f"Running: a={a}, b={b}, activation={act}, loss={loss_fn}")
#                     final_loss = train_bidirectional_lstm(a, b, act, loss_fn, 200)
#                     writer.writerow([a, b, act, loss_fn, final_loss])

In [17]:
# with open('results_cnn_lstm.csv', mode='w', newline='') as file:
#     writer = csv.writer(file)
#     writer.writerow(['a', 'b', 'activation', 'loss_function', 'final_loss'])  # Header

#     for a in range(2, 4):
#         for b in range(1, 4):
#             for c in range (1, 3):
#                 for act in activations:
#                     for loss_fn in losses:
#                         print(f"Running: a={a}, b={b}, activation={act}, loss={loss_fn}")
#                         final_loss = train_cnn_lstm(a, b, c, act, loss_fn, 200)
#                         writer.writerow([a, b, act, loss_fn, final_loss])

In [18]:
# with open('results_bidirectional_stacked_lstm.csv', mode='w', newline='') as file:
#     writer = csv.writer(file)
#     writer.writerow(['a', 'b', 'activation', 'loss_function', 'final_loss'])  # Header

#     for a in range(1, 4):
#         for b in range(1, 3):
#                 for act in activations:
#                     for loss_fn in losses:
#                         print(f"Running: a={a}, b={b}, activation={act}, loss={loss_fn}")
#                         final_loss = train_bidirectional_stacked_lstm(a, b, act, loss_fn, 200)
#                         writer.writerow([a, b, act, loss_fn, final_loss])

In [19]:
# with open('results_bidirectional_cnn_lstm.csv', mode='w', newline='') as file:
#     writer = csv.writer(file)
#     writer.writerow(['a', 'b', 'activation', 'loss_function', 'final_loss'])  # Header

#     for a in range(2, 4):
#         for b in range(1, 4):
#             for c in range(1, 3):
#                 for act in activations:
#                     for loss_fn in losses:
#                         print(f"Running: a={a}, b={b}, activation={act}, loss={loss_fn}")
#                         final_loss = train_bidirectional_cnn_lstm(a, b, c, act, loss_fn, 200)
#                         writer.writerow([a, b, act, loss_fn, final_loss])